In [ ]:
import sys
import os
import itertools

base_path = os.path.dirname(os.path.dirname(os.path.abspath(".")))
sys.path.append(base_path)

import torch.optim as optim
import numpy as np
from tqdm import tqdm
import logging


from utils.config import img_param_init, set_random_seed
from utils.dataload import DomainDataset, get_data
from model.clip import ClipModelMA, Adapter, Client
from process import train_client, communicate, test_server, test_client, fetch
class argparser():
   dataset = 'officehome'
   root_dir='/mnt/sda/zd/data/splitdata_new/'
   net='ViT-B/16'
   seed=0
   logdir=None
   device="cuda:0"
   n_clients=10
   n_experts=5
   inner_iter=10
   n_task=10
   # data setting parameter
   datapercent=8e-1
   batch=100
   # optimizer parameter
   lr=2e-3
   beta1=0.9
   beta2=0.98
   eps=1e-6
   weight_decay=0.2
import logging
def init():
    args = argparser()
    if args.logdir is not None:
        logging.basicConfig(filename=args.logdir, level=logging.DEBUG)
    else:
        logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
    args.random_state = np.random.RandomState(1)
    set_random_seed(args.seed)
    args = img_param_init(args)  # init the dataset parameters(domains and classnum)
    args.thresh = 1e-4
    return args

In [ ]:
print(1)

In [ ]:
    
def init():
    args = argparser()
    if args.logdir is not None:
        logging.basicConfig(filename=args.logdir, level=logging.DEBUG)
    else:
        logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
    args.random_state = np.random.RandomState(1)
    set_random_seed(args.seed)
    args = img_param_init(args)  # init the dataset parameters(domains and classnum)
    args.thresh = 1e-4
    return args

In [ ]:
def freeze_param(model):
    for name, param in model.named_parameters():
        param.requires_grad = False
args = init()

args.dataset = "officehome"
args.root_dir = "/mnt/sda/zd/data/OF10-10-1000-2023/"

logging.info("Argument init successful!")
server_model: ClipModelMA = ClipModelMA(args.net, n_experts=args.n_experts, device=args.device)  # load the server data
dataloader: DomainDataset = get_data(args.dataset)(args, server_model.preprocess)
train_loaders, test_loaders, labels = dataloader.get_dataloader(8)
assert len(train_loaders) == args.n_clients, "This just mention you to make sure the n_clients \
                                            equals to the datasplit client number."
server_model.labels = labels
server_model.init_prompt()
server_model.init_MoE()
freeze_param(server_model.model)
logging.info("Data init successful!")



In [ ]:
import torch
server_model.MoE.gating.load_state_dict(torch.load("save/gating8.pkl"))
server_model.MoE.gating.to(args.device)

In [ ]:
# from torch.utils.data import TensorDataset, DataLoader
clients_data = [Client("ViT-B/16", args.device) for _ in range(10)]
with torch.no_grad():
    all_feature = []
    for idx, client_loader in tqdm(enumerate(train_loaders)):
        temp_hook = server_model.model.visual.transformer.resblocks[5].register_forward_hook(clients_data[idx].extract_feature())
        for image,_,_ in client_loader:
            server_model.model.encode_image(image.to(args.device))
        temp_hook.remove()

        single_feature = []
        temp_data = torch.tensor(clients_data[idx].preprocess(), dtype=torch.float)
        temp_array = np.array([0 for i in range(args.n_experts)], dtype=np.float64)
        for i, data in enumerate(temp_data):
            res = server_model.MoE.gating(data.to(args.device))
            # print(np.array(res.cpu()))
            temp_array += np.array(res.cpu(), dtype=np.float64)
            if (i%20) == 0:
                single_feature.append(temp_array/20)
                temp_array = np.array([0 for i in range(args.n_experts)],dtype=np.float64)
        all_feature.append(np.stack(single_feature))
all_feature = np.vstack(all_feature)
all_feature.shape    


In [ ]:
np.save("OF.dat", all_feature)

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import itertools

label = [0,1,2,3,0,1,2,3,0,1]
labels = itertools.chain.from_iterable([[i]*25 for i in label ])

print("执行t-SNE降维")
tsne = TSNE(n_components=2, perplexity=50, learning_rate=1, random_state=42)
tsne_result = tsne.fit_transform(all_feature)

print("可视化数据并使用标签着色")
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=list(labels))  # 使用'viridis'颜色地图
plt.title("t-SNE Visualization with Labels")
# plt.colorbar()
plt.show()

# DoaminNet

In [ ]:
import sys
import os
import itertools

base_path = os.path.dirname(os.path.dirname(os.path.abspath(".")))
sys.path.append(base_path)

import torch.optim as optim
import numpy as np
from tqdm import tqdm
import logging


from utils.config import img_param_init, set_random_seed
from utils.dataload import DomainDataset, get_data
from model.clip import ClipModelMA, Adapter, Client
from process import train_client, communicate, test_server, test_client, fetch
class argparser():
   dataset = 'domainnetsub'
   root_dir='/mnt/sda/zd/data/DN20-10-2000-2023'
   net='ViT-B/16'
   seed=0
   logdir=None
   device="cuda:0"
   n_clients=20
   n_experts=8
   inner_iter=10
   n_task=10
   # data setting parameter
   datapercent=8e-1
   batch=100
   # optimizer parameter
   lr=2e-3
   beta1=0.9
   beta2=0.98
   eps=1e-6
   weight_decay=0.2
import logging
def init():
    args = argparser()
    if args.logdir is not None:
        logging.basicConfig(filename=args.logdir, level=logging.DEBUG)
    else:
        logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
    args.random_state = np.random.RandomState(1)
    set_random_seed(args.seed)
    args = img_param_init(args)  # init the dataset parameters(domains and classnum)
    args.thresh = 1e-4
    return args

In [ ]:
    
def init():
    args = argparser()
    if args.logdir is not None:
        logging.basicConfig(filename=args.logdir, level=logging.DEBUG)
    else:
        logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
    args.random_state = np.random.RandomState(1)
    set_random_seed(args.seed)
    args = img_param_init(args)  # init the dataset parameters(domains and classnum)
    args.thresh = 1e-4
    return args

In [ ]:
def freeze_param(model):
    for name, param in model.named_parameters():
        param.requires_grad = False
args = init()

args.dataset = "domainnetsub"
args.root_dir = "/mnt/sda/zd/data/DN20-10-2000-2023"

logging.info("Argument init successful!")
server_model: ClipModelMA = ClipModelMA(args.net, n_experts=args.n_experts, device=args.device)  # load the server data
dataloader: DomainDataset = get_data(args.dataset)(args, server_model.preprocess)
train_loaders, test_loaders, labels = dataloader.get_dataloader(8)
assert len(train_loaders) == args.n_clients, "This just mention you to make sure the n_clients \
                                            equals to the datasplit client number."
server_model.labels = labels
server_model.init_prompt()
server_model.init_MoE()
freeze_param(server_model.model)
logging.info("Data init successful!")



In [ ]:
import torch
server_model.MoE.gating.load_state_dict(torch.load("save/gatingDN8.pkl"))
server_model.MoE.gating.to(args.device)

In [ ]:
# from torch.utils.data import TensorDataset, DataLoader
clients_data = [Client("ViT-B/16", args.device) for _ in range(20)]
with torch.no_grad():
    all_feature_DN = []
    for idx, client_loader in tqdm(enumerate(train_loaders)):
        temp_hook = server_model.model.visual.transformer.resblocks[8].register_forward_hook(clients_data[idx].extract_feature())
        for image,_,_ in client_loader:
            server_model.model.encode_image(image.to(args.device))
        temp_hook.remove()

        single_feature = []
        temp_data = torch.tensor(clients_data[idx].preprocess(), dtype=torch.float)
        temp_array = np.array([0 for i in range(args.n_experts)], dtype=np.float64)
        for i, data in enumerate(temp_data):
            res = server_model.MoE.gating(data.to(args.device))
            # print(np.array(res.cpu()))
            temp_array += np.array(res.cpu(), dtype=np.float64)
            if (i%20) == 0:
                single_feature.append(temp_array/20)
                temp_array = np.array([0 for i in range(args.n_experts)],dtype=np.float64)
        all_feature_DN.append(np.stack(single_feature))
all_feature_DN = np.vstack(all_feature_DN)
all_feature_DN.shape

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import itertools

label_DN = [0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1]
labels_DN = itertools.chain.from_iterable([[i]*25 for i in label_DN ])

print("执行t-SNE降维")
tsne = TSNE(n_components=2, perplexity=40, learning_rate=100, random_state=42)
tsne_result = tsne.fit_transform(all_feature_DN)

print("可视化数据并使用标签着色")
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=list(labels_DN))  # 使用'viridis'颜色地图
# plt.title("t-SNE Visualization with Labels")
# plt.colorbar()
plt.show()

In [ ]:
# labels_DN
# labels
# all_feature
# all_feature_DN
import matplotlib.pyplot as plt
import scienceplots
plt.style.use(['no-latex', 'scatter'])
import numpy as np
# palette = ["#287246", "#B1C44D", "#E7C736", "#F7BF63", "#EEB0AF","#963460","#631F66", "#01899D"]
# label_DN =  ["#287246", "#B1C44D", "#E7C736", "#F7BF63", "#EEB0AF","#963460","#287246", "#B1C44D", "#E7C736", "#F7BF63", "#EEB0AF","#963460","#287246", "#B1C44D", "#E7C736", "#F7BF63", "#EEB0AF","#963460","#287246", "#B1C44D"]
# label = ["#287246", "#B1C44D", "#E7C736", "#F7BF63", "#287246", "#B1C44D", "#E7C736", "#F7BF63","#287246", "#B1C44D"]
label_DN = ["#14517C", "#2F7FC1", "#E7EFFA" ,"#96C37D", "#F3D266", "#D8383A", "#14517C", "#2F7FC1", "#E7EFFA" ,"#96C37D", "#F3D266", "#D8383A", "#14517C", "#2F7FC1", "#E7EFFA" ,"#96C37D", "#F3D266", "#D8383A", "#14517C", "#2F7FC1"]  
label = ["#2F7FC1", "#96C37D", "#F3D266", "#D8383A", "#2F7FC1", "#96C37D", "#F3D266", "#D8383A","#2F7FC1", "#96C37D"]
palette = ["#14517C", "#2F7FC1", "#E7EFFA" ,"#96C37D", "#F3D266", "#D8383A"]
plt.rcParams.update({'font.family': 'serif'})

labels_DN = list(itertools.chain.from_iterable([[i]*25 for i in label_DN ]))
labels = list(itertools.chain.from_iterable([[i]*25 for i in label ]))


plt.rcParams.update({'font.family': 'serif'})
tsne_OF = TSNE(n_components=2, perplexity=40, learning_rate=100, random_state=42)
tsne_result_OF = tsne_OF.fit_transform(all_feature)
tsne_DN = TSNE(n_components=2, perplexity=10, learning_rate=400, random_state=42)
tsne_result_DN = tsne_DN.fit_transform(all_feature_DN)
fig, axes = plt.subplots(2, 1, figsize=(5, 4))

axes[0].scatter(tsne_result_OF[:, 0], tsne_result_OF[:, 1], c=list(labels), s=7)
axes[0].set_title('(a) Office-Home')


# Plot for DN
axes[1].scatter(tsne_result_DN[:, 0], tsne_result_DN[:, 1], c=list(labels_DN), s=5)
axes[1].set_title('(b) DomainNet')


plt.tight_layout()
plt.savefig("tsne.pdf")



In [ ]:
tsne_DN = TSNE(n_components=2, perplexity=40, learning_rate=100, random_state=42)
tsne_result_DN = tsne_DN.fit_transform(all_feature_DN)
plt.scatter(tsne_result_DN[:, 0], tsne_result_DN[:, 1], c=list(labels_DN))  # 使用'viridis'颜色地图
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import itertools
from sklearn.manifold import TSNE
import numpy as np

# 生成分类标签
categories = ['a', 'b', 'c', 'd']
categories_DN = ['a', 'b', 'c', 'd', 'e', 'f']
label_DN = [0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5,0,1]
label = [0,1,2,3,0,1,2,3,0,1]
labels_DN = list(itertools.chain.from_iterable([[i]*25 for i in label_DN ]))
labels = list(itertools.chain.from_iterable([[i]*25 for i in label ]))
# 运行 t-SNE
tsne_OF = TSNE(n_components=2, perplexity=40, learning_rate=100, random_state=42)
tsne_result_OF = tsne_OF.fit_transform(all_feature)

tsne_DN = TSNE(n_components=2, perplexity=10, learning_rate=400, random_state=42)
tsne_result_DN = tsne_DN.fit_transform(all_feature_DN)
plt.show()
# 创建子图
fig, axes = plt.subplots(2, 1, figsize=(5, 4)) # 调整figsize以留出空间放置图例

# 绘制散点图并为每个类别添加一个代表点以用于图例
for i, category in enumerate(categories):
    subset = tsne_result_OF[np.array(list(labels)) == i]
    axes[0].scatter(subset[:, 0], subset[:, 1], label=category, s=7)
axes[0].set_title('(a) Office-Home')
# axes[0].legend()

for i, category in enumerate(categories_DN):
    subset = tsne_result_DN[np.array(list(labels_DN)) == i]
    axes[1].scatter(subset[:, 0], subset[:, 1], label=category, s=5)
axes[1].set_title('(b) DomainNet')
# axes[1].legend()
plt.tight_layout()
# 显示图形
plt.show()

In [2]:
import sys
import os
import itertools

base_path = os.path.dirname(os.path.dirname(os.path.abspath(".")))
sys.path.append(base_path)

import torch.optim as optim
import numpy as np
from tqdm import tqdm
import logging


from utils.config import img_param_init, set_random_seed
from utils.dataload import DomainDataset, get_data
from model.clip import ClipModelMA, Adapter, Client
from process import train_client, communicate, test_server, test_client, fetch
class argparser():
   dataset = 'adaptiope'
   root_dir='/mnt/sda/zd/data/AD10-10-1000-2023'
   net='ViT-B/16'
   seed=0
   logdir=None
   device="cuda:0"
   n_clients=10
   n_experts=3
   inner_iter=10
   n_task=10
   # data setting parameter
   datapercent=8e-1
   batch=100
   # optimizer parameter
   lr=2e-3
   beta1=0.9
   beta2=0.98
   eps=1e-6
   weight_decay=0.2
import logging
def init():
    args = argparser()
    if args.logdir is not None:
        logging.basicConfig(filename=args.logdir, level=logging.DEBUG)
    else:
        logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
    args.random_state = np.random.RandomState(1)
    set_random_seed(args.seed)
    args = img_param_init(args)  # init the dataset parameters(domains and classnum)
    args.thresh = 1e-4
    return args
    
def init():
    args = argparser()
    if args.logdir is not None:
        logging.basicConfig(filename=args.logdir, level=logging.DEBUG)
    else:
        logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
    args.random_state = np.random.RandomState(1)
    set_random_seed(args.seed)
    args = img_param_init(args)  # init the dataset parameters(domains and classnum)
    args.thresh = 1e-4
    return args
def freeze_param(model):
    for name, param in model.named_parameters():
        param.requires_grad = False
args = init()

logging.info("Argument init successful!")
server_model: ClipModelMA = ClipModelMA(args.net, n_experts=args.n_experts, device=args.device)  # load the server data
dataloader: DomainDataset = get_data(args.dataset)(args, server_model.preprocess)
train_loaders, test_loaders, labels = dataloader.get_dataloader(7)
assert len(train_loaders) == args.n_clients, "This just mention you to make sure the n_clients \
                                            equals to the datasplit client number."
server_model.labels = labels
server_model.init_prompt()
server_model.init_MoE()
freeze_param(server_model.model)
logging.info("Data init successful!")
import torch
server_model.MoE.gating.load_state_dict(torch.load("save/gatingAD7.pkl"))
server_model.MoE.gating.to(args.device)

Argument init successful!
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/client0/task7/real_life
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/client1/task7/product_images
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/client2/task7/synthetic
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/client3/task7/real_life
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/client4/task7/product_images
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/client5/task7/synthetic
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/client6/task7/real_life
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/client7/task7/product_images
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/client8/task7/synthetic
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/client9/task7/real_life
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/test/synthetic with ['synthetic', 'real_life', 'product_images']
loading data in /mnt/sda/zd/data/AD10-10-1000-2023/test/real_life with ['synthetic', 'real_li

Sequential(
  (0): Linear(in_features=768, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=512, out_features=3, bias=True)
)

In [3]:
# from torch.utils.data import TensorDataset, DataLoader
clients_data = [Client("ViT-B/16", args.device) for _ in range(10)]
with torch.no_grad():
    all_feature_DN = []
    for idx, client_loader in tqdm(enumerate(train_loaders)):
        temp_hook = server_model.model.visual.transformer.resblocks[5].register_forward_hook(clients_data[idx].extract_feature())
        for image,_,_ in client_loader:
            server_model.model.encode_image(image.to(args.device))
        temp_hook.remove()

        single_feature = []
        temp_data = torch.tensor(clients_data[idx].preprocess(), dtype=torch.float)
        temp_array = np.array([0 for i in range(args.n_experts)], dtype=np.float64)
        for i, data in enumerate(temp_data):
            res = server_model.MoE.gating(data.to(args.device))
            # print(np.array(res.cpu()))
            temp_array += np.array(res.cpu(), dtype=np.float64)
            if (i%20) == 0:
                single_feature.append(temp_array/20)
                temp_array = np.array([0 for i in range(args.n_experts)],dtype=np.float64)
        all_feature_DN.append(np.stack(single_feature))
all_feature_DN = np.vstack(all_feature_DN)
all_feature_DN.shape

0it [00:00, ?it/s]

DEBUG:PIL.TiffImagePlugin:tag: Make (271) - type: string (2) Tag Location: 22 - Data Location: 122 - value: b'SAMSUNG\x00'
DEBUG:PIL.TiffImagePlugin:tag: Model (272) - type: string (2) Tag Location: 34 - Data Location: 130 - value: b'GT-I9100\x00'
DEBUG:PIL.TiffImagePlugin:tag: XResolution (282) - type: rational (5) Tag Location: 46 - Data Location: 140 - value: b'\x00\x00\x00H\x00\x00\x00\x01'
DEBUG:PIL.TiffImagePlugin:tag: YResolution (283) - type: rational (5) Tag Location: 58 - Data Location: 148 - value: b'\x00\x00\x00H\x00\x00\x00\x01'
DEBUG:PIL.TiffImagePlugin:tag: ResolutionUnit (296) - type: short (3) - value: b'\x00\x02'
DEBUG:PIL.TiffImagePlugin:tag: Software (305) - type: string (2) Tag Location: 82 - Data Location: 156 - value: <table: 41 bytes>
DEBUG:PIL.TiffImagePlugin:tag: DateTime (306) - type: string (2) Tag Location: 94 - Data Location: 198 - value: b'2012:10:01 11:57:23\x00'
DEBUG:PIL.TiffImagePlugin:tag: YCbCrPositioning (531) - type: short (3) - value: b'\x00\x01'

2it [00:34, 16.74s/it]

DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'pHYs' 41 9
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 62 66
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 140 24
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 176 16
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 204 9
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 225 13
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 250 11
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 273 19
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 304 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'pHYs' 41 9
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 62 50
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 124 24
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 160 16
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 188 9
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 209 13
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 234 11
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 257 19
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 288 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePl

3it [01:19, 29.70s/it]

...

DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'pHYs' 41 9
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 62 56
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 130 24
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 166 16
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 194 9
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 215 13
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 240 11
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 263 19
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 294 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'pHYs' 41 9
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 62 54
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 128 24
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 164 16
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 192 9
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 213 13
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 238 11
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 261 19
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 292 8192
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePl

9it [04:01, 30.00s/it]

DEBUG:PIL.TiffImagePlugin:tag: Make (271) - type: string (2) Tag Location: 22 - Data Location: 194 - value: b'Canon\x00'
DEBUG:PIL.TiffImagePlugin:tag: Model (272) - type: string (2) Tag Location: 34 - Data Location: 200 - value: b'Canon PowerShot S400\x00'
DEBUG:PIL.TiffImagePlugin:tag: XResolution (282) - type: rational (5) Tag Location: 46 - Data Location: 222 - value: b'\xb4\x00\x00\x00\x01\x00\x00\x00'
DEBUG:PIL.TiffImagePlugin:tag: YResolution (283) - type: rational (5) Tag Location: 58 - Data Location: 230 - value: b'\xb4\x00\x00\x00\x01\x00\x00\x00'
DEBUG:PIL.TiffImagePlugin:tag: ResolutionUnit (296) - type: short (3) - value: b'\x02\x00'
DEBUG:PIL.TiffImagePlugin:tag: DateTime (306) - type: string (2) Tag Location: 82 - Data Location: 238 - value: b'2007:03:15 14:31:25\x00'
DEBUG:PIL.TiffImagePlugin:tag: YCbCrPositioning (531) - type: short (3) - value: b'\x01\x00'
DEBUG:PIL.TiffImagePlugin:tag: unknown (4097) - type: short (3) - value: b'\xe0\x08'
DEBUG:PIL.TiffImagePlugin:ta

10it [04:23, 26.32s/it]


(500, 3)

In [4]:
np.save("ad.npy", all_feature_DN)